In [ ]:
from collections import Counter
from itertools import combinations
import time
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue  #節點元素名稱，在構造時初始化爲給定值
        self.count = numOccur   # 出現次數，在構造時初始化爲給定值
        self.nodeLink = None   # 指向下一個相似節點的指針，默認爲None
        self.parent = parentNode   # 指向父節點的指針，在構造時初始化爲給定值
        self.children = {}  # 指向子節點的字典，以子節點的元素名稱爲鍵，指向子節點的指針爲值，初始化爲空字典

    # 增加節點的出現次數值
    def inc(self, numOccur):
        self.count += numOccur

    # 輸出節點和子節點的FP樹結構
    def disp(self, ind=1):
        #print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

# =======================================================構建FP樹==================================================

# 對不是第一個出現的節點，更新頭指針塊。就是添加到相似元素鏈表的尾部
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 根據一個排序過濾後的頻繁項更新FP樹
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 有該元素項時計數值+1
        inTree.children[items[0]].inc(count)
    else:
        # 沒有這個元素項時創建一個新節點
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新頭指針表或前一個相似元素項節點的指針指向新節點
        if headerTable[items[0]][1] == None:  # 如果是第一次出現，則在頭指針表中增加對該節點的指向
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:
        # 對剩下的元素項迭代調用updateTree函數
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 主程序。創建FP樹。dataSet爲事務集，爲一個字典，鍵爲每個事物，值爲該事物出現的次數。minSup爲最低支持度
def createTree(dataSet, minSup):
    # 第一次遍歷數據集，創建頭指針表
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不滿足最小支持度的元素項
    keys = list(headerTable.keys()) # 因爲字典要求在迭代中不能修改，所以轉化爲列表
    for k in keys:
        if headerTable[k] < minSup:
            del(headerTable[k])
    # 空元素集，返回空
    freqItemSet = frozenset(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    # 增加一個數據項，用於存放指向相似元素項指針
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # 每個鍵的值，第一個爲個數，第二個爲下一個節點的位置
    retTree = treeNode('Null Set', 1, None) # 根節點
    # 第二次遍歷數據集，創建FP樹
    for tranSet, count in dataSet.items():
        localD = {} # 記錄頻繁1項集的全局頻率，用於排序
        for item in tranSet:
            if item in freqItemSet:   # 只考慮頻繁項
                localD[item] = headerTable[item][0] # 注意這個[0]，因爲之前加過一個數據項
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: (p[1],int(p[0])), reverse=True)]  # (p[1],int(p[0])
            updateTree(orderedItems, retTree, headerTable, count)  # populate tree with ordered freq itemset
    return retTree, headerTable

# =================================================查找元素條件模式基===============================================

# 直接修改prefixPath的值，將當前節點leafNode添加到prefixPath的末尾，然後遞歸添加其父節點。
# prefixPath就是一條從treeNode（包括treeNode）到根節點（不包括根節點）的路徑
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

# 爲給定元素項生成一個條件模式基（前綴路徑）。basePet表示輸入的頻繁項，treeNode爲當前FP樹中對應的第一個節點
# 函數返回值即爲條件模式基condPats，用一個字典表示，鍵爲前綴路徑，值爲計數值。
def findPrefix(basePat, treeNode):
    condPats = {}  # 存儲條件模式基
    while treeNode != None:
        prefixPath = []  # 用於存儲前綴路徑
        ascendTree(treeNode, prefixPath)  # 生成前綴路徑
        if len(prefixPath) >= 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count  # 出現的數量就是當前葉子節點的數量
        treeNode = treeNode.nodeLink  # 遍歷下一個相同元素
    return condPats

# =================================================遞歸查找頻繁項集===============================================
# 根據事務集獲取FP樹和頻繁項。
# 遍歷頻繁項，生成每個頻繁項的條件FP樹和條件FP樹的頻繁項
# 這樣每個頻繁項與他條件FP樹的頻繁項都構成了頻繁項集

# inTree和headerTable是由createTree()函數生成的事務集的FP樹。
# minSup表示最小支持度。
# preFix請傳入一個空集合（set([])），將在函數中用於保存當前前綴。
# freqItemList請傳入一個空列表（[]），將用來儲存生成的頻繁項集。
def finalTree(inTree,headerTable,minSup,preFix,freqItemList,ans):
    # 對頻繁項按出現的數量進行排序
    sorted_headerTable = sorted(headerTable.items(), key=lambda p: p[1][0])  #返回重新排序的列表。每個元素是一個元組，[（key,[num,treeNode],()）
    bigL = [v[0] for v in sorted_headerTable]  # 獲取頻繁項
    for basePat in bigL:
      newFreqSet = preFix.copy()  # 新的頻繁項集
      newFreqSet.add(basePat)     # 當前前綴添加一個新元素
      if len(newFreqSet)<6:     
        freqItemList.append(newFreqSet)  # 所有的頻繁項集列表
        positive = findPrefix(basePat, headerTable[basePat][1])  # 獲取條件模式基。就是basePat元素的所有前綴路徑。它像一個新的事務集
        sum = 0
        for i in positive.values():
          sum += i
        ans.update({frozenset(newFreqSet):sum})
        myTree, myHead = createTree(positive, minSup)  # 創建條件FP樹
        if myHead != None and len(preFix) < 4:# 用於測試            
            finalTree(myTree, myHead, minSup, newFreqSet, freqItemList,ans)  # 遞歸直到不再有元素

# 生成數據集
def loadDat():
    simpDat =[line.split() for line in open("/mushroom (2).dat",'r').readlines()]
    return simpDat

# 將數據集轉化爲目標格式
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
      retDict[frozenset(trans)] = 1
    return retDict

if __name__=='__main__':
    start=time.time()
    minSup = 813
    simpDat = loadDat()  # 加載數據集
    initSet = createInitSet(simpDat)  # 轉化爲符合格式的事務集
    mytree, myHeaderTab = createTree(initSet, minSup)  # 形成FP樹
    freqItems = []  # 用於存儲頻繁項集
    ans={}
    finalTree(mytree,myHeaderTab,minSup,set([]),freqItems,ans)  # 獲取頻繁項集
L1=0
L2=0
L3=0
L4=0
L5=0
count = 0
for i in freqItems:
  if(len(i)==1):
    L1+=1
  elif(len(i)==2):
    L2+=1
  elif(len(i)==3):
    L3+=1
  elif(len(i)==4):
    L4+=1
  elif(len(i)==5):
    L5+=1
for freqitem in ans:
  yyds=[c for n in range(1,len(freqitem)) for c in combinations(freqitem,n)]
  for yy in yyds:
      if(float(ans[frozenset(freqitem)] / ans[frozenset(yy)]) >= 0.8):
        count += 1
end=time.time()
print("Total Execution Time: {}s".format(end-start))
print("|L^1|= {}".format(L1))
print("|L^2|= {}".format(L2))
print("|L^3|= {}".format(L3))
print("|L^4|= {}".format(L4))
print("|L^5|= {}".format(L5))
print("All Association rules: {}".format(count))

Total Execution Time: 4.089094161987305s
|L^1|= 56
|L^2|= 763
|L^3|= 4593
|L^4|= 16150
|L^5|= 38800
All Association rules: 394175
